<a href="https://colab.research.google.com/github/ohjisu320/study_data_analytics/blob/main/docs/quests/CDAs/CDA_fruitstore_continuous.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한글 폰트 설치
- !apt-get install -y fonts-nanum
- !fc-cache -fv
- !rm ~/.cache/matplotlib -rf

  - 설치 후 colab ruddn runtime > restart session 필요

In [1]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [2]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

# 전처리
- 결측치 처리
- column 생성

In [28]:
df_gro =pd.read_csv('fruitstore_saleshistory.csv')

In [29]:
condition = "물품중분류 =='과일'"
df_fruit = df_gro.query(condition)
df_fruit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15402 entries, 0 to 705478
Data columns (total 20 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   공급일자      15402 non-null  object 
 1   요일        15402 non-null  object 
 2   공급월       15402 non-null  int64  
 3   공급주차      15402 non-null  int64  
 4   회원번호      15402 non-null  int64  
 5   조합원상태     15402 non-null  object 
 6   물품대분류     15402 non-null  object 
 7   물품중분류     15402 non-null  object 
 8   물품소분류     15402 non-null  object 
 9   물품명       15402 non-null  object 
 10  구매수량      15402 non-null  float64
 11  주소-구      15402 non-null  object 
 12  주소-동      15402 non-null  object 
 13  성별        15402 non-null  object 
 14  연령        15402 non-null  float64
 15  연령대       15402 non-null  object 
 16  구매금액      15402 non-null  int64  
 17  구매매장      15402 non-null  object 
 18  반품_원거래일자  325 non-null    float64
 19  구매시각      15402 non-null  object 
dtypes: float64(3), int64(4), ob

In [30]:
df_fruit['구매수량'].unique()

array([  1.  ,  -1.  ,   5.  ,   2.  ,   0.25,   0.5 ,   3.  ,  -1.5 ,
        -0.5 ,   0.  ,  -2.  ,  10.  , -10.  ,   4.  ,  -5.  ,   6.  ,
         0.4 ,   0.2 ,   0.6 ,   0.1 ,   0.32,   0.44,   1.5 ,   0.31,
        -0.44,  -0.7 ,  -4.  ,  -3.  ])

In [31]:
df_fruit['상품단가'] = df_fruit['구매금액']/df_fruit['구매수량']

<ipython-input-31-2393ae11a27c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fruit['상품단가'] = df_fruit['구매금액']/df_fruit['구매수량']


## 판매된 수량(X)이 각 과일 유형의 단가(Y)와 어떤 관계가 있습니까?
- 분석 방법: 수량과 가격 사이의 관계의 강도와 방향을 측정하기 위한 피어슨 또는 스피어만


## 정규분포 여부 확인
- 5천개 이상이기에 anderson noramlity test로 확인
  - 정규분포 -> pearsonr

In [32]:
df_fruit = df_fruit.rename(columns={'주소-구': '주소_구'})
df_fruit = df_fruit.rename(columns={'주소-동': '주소_동'})

In [33]:
import numpy as np
pd.pivot_table(data=df_fruit[['물품소분류', '구매수량']]
               , index='구매수량', columns='물품소분류', aggfunc=len)

물품소분류,건대추,곶감,귤(비가림),귤(조생),귤만감,냉동건조과일,단감,레몬,배,복숭아,블루베리,사과,산딸기,자두,참다래,청매실,홍시용감(대봉),황매실
구매수량,,,,,,,,,,,,,,,,,,
-10.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
-4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
-3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-2.00,NaN,4.0,NaN,NaN,2.0,NaN,5.0,NaN,NaN,NaN,1.0,10.0,2.0,NaN,NaN,1.0,NaN,2.0
-1.50,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-1.00,12.0,7.0,3.0,16.0,40.0,NaN,9.0,2.0,38.0,NaN,10.0,125.0,15.0,NaN,26.0,3.0,NaN,3.0
-0.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-0.50,NaN,NaN,NaN,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_fruit

In [34]:
df_fruit['상품단가'] = df_fruit['구매금액']/df_fruit['구매수량']

In [36]:
stats.anderson( df_fruit['상품단가']) # 정규분포임 확인

AndersonResult(statistic=nan, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=22775.750947093016, scale=12469.689431275572)
 success: False
 message: 'Optimization converged to parameter values that are inconsistent with the data.')

### 정규분포 -> pearsonr

In [47]:
df_fruit_number_price = df_fruit[['상품단가', '구매수량']].dropna()

In [48]:
stats.pearsonr(df_fruit_number_price['상품단가'], df_fruit_number_price['구매수량'])

PearsonRResult(statistic=-0.0876561773808418, pvalue=1.3145251519662877e-27)

In [49]:
# 두 값의 상관계수 확인
correlation = df_fruit_number_price[['상품단가', '구매수량']].corr()
print(correlation)


          상품단가      구매수량
상품단가  1.000000 -0.087656
구매수량 -0.087656  1.000000


### 분석 결과
- 통계 결과 : 귀무가설 기각, 대립가설 채택(상품단가와 구매수량의 연관성이 있음)
- 사실 결과 : 상품 단가와 구매수량은 음의 상관관계를 가지나, 기울기가 매우 낮으므로, 상관관계가 약하다. 따라서 구매수량에 강력한 영향을 미치는 다른 요소를 분석할 필요성이 있다.  